In [1]:
import tensorflow as tf
print(tf.__version__)

1.13.1


## Simple graph and execution

In [2]:
x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y + 2

sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)

print(result)

sess.close()

Instructions for updating:
Colocations handled automatically by placer.
42


In [3]:
# Create an auto-closed session

with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

print(result)

42


In [4]:
# Global initializer

init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    result = f.eval()

print(result)

42


In [5]:
# Interactive session
# When instantiated, automatically set as the default session

sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close()

42


In [6]:
# Managing nodes in graphs

# Nodes are automatically added to default graph
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [7]:
# Can make a graph as the default temporarily and add nodes to it
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

print(x2.graph is graph)
print(x2.graph is tf.get_default_graph())

True
False


## Linear Regression

In [18]:
import numpy as np
from sklearn.datasets import fetch_california_housing

In [9]:
# Fetch the housing data and add a bias term to the front (column of m 1s)
housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

print(housing_data_plus_bias.shape)

(20640, 9)


In [13]:
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
XT = tf.transpose(X)
theta = tf.linalg.matmul(tf.linalg.matmul(tf.matrix_inverse(tf.linalg.matmul(XT, X)), XT), y) # Normal equation

init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    theta_value = sess.run(theta)

print(theta_value)

[[-3.6959320e+01]
 [ 4.3698898e-01]
 [ 9.4245886e-03]
 [-1.0791138e-01]
 [ 6.4842808e-01]
 [-3.9986235e-06]
 [-3.7866351e-03]
 [-4.2142656e-01]
 [-4.3467718e-01]]


# Gradient Descent

In [15]:
# Set hyperparameters
n_epochs = 1000
learning_rate = 0.01

In [33]:
# Scale the housing data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_housing_data_plus_bias = scaler.fit_transform(housing_data_plus_bias)

In [49]:
# Set up the graph ops

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
#gradients = 2/m * tf.matmul(tf.transpose(X), error) # manual differentiation
#gradients = tf.gradients(mse, [theta])[0] # automatic differentiation
#optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate) # Use TF built-in GD optimizer
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9) # TF Momentum optimizer
training_op = optimizer.minimize(mse) # Using the optimizer for the training operation
#training_op = tf.assign(theta, theta - learning_rate * gradients) # manual training operation

In [50]:
# Execute the graph

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Epoch', epoch, 'MSE = ', mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

Epoch 0 MSE =  6.547937
Epoch 100 MSE =  4.8137703
Epoch 200 MSE =  4.803608
Epoch 300 MSE =  4.8032823
Epoch 400 MSE =  4.8032575
Epoch 500 MSE =  4.8032546
Epoch 600 MSE =  4.803253
Epoch 700 MSE =  4.803254
Epoch 800 MSE =  4.8032546
Epoch 900 MSE =  4.8032537


In [51]:
print(best_theta)

[[ 0.44966483]
 [ 0.8296246 ]
 [ 0.11875261]
 [-0.26553705]
 [ 0.30570474]
 [-0.00450274]
 [-0.03932647]
 [-0.89987403]
 [-0.8705295 ]]
